In [40]:
import requests
from urllib.parse import quote
from alive_progress import alive_it

## List all pages in category

In [41]:
API_NAME = "weapons"
FORMAT = "weapon"

In [42]:
r = requests.get(f"https://api.star-citizen.wiki/api/v2/{API_NAME}/?limit=10000&locale=en_EN").json()["data"]

In [43]:
r = [(component["uuid"], component["name"]) for component in r if component["name"] != "<= PLACEHOLDER =>" and (component["uuid"] is not None)]

In [44]:
r

[('9e8d6cf1-17d6-453e-bfe3-44529969eebf', 'Demon Fang Combat Knife'),
 ('699db405-4ca1-4993-ba58-a9f90001ae6b', 'Scourge Railgun'),
 ('517834f4-9343-4aaa-907c-6d911d221936', 'Scourge "Nightstalker" Railgun'),
 ('35e6ac2c-6734-47d7-bc9a-b7e5084dc48e', 'Scourge "Desert Shadow" Railgun'),
 ('3666c8ac-d552-4006-8c47-7e8f773b11a4', 'Animus Missile Launcher'),
 ('a6688dce-f8c7-4b73-b657-bd1e17f951d3', 'GP-33 MOD Grenade Launcher'),
 ('5d8e354f-5f52-467a-9546-9c5a981f4257',
  'GP-33 MOD "Thunderclap" Grenade Launcher'),
 ('e4a479eb-e58d-4fba-8d48-037af894b7b9',
  'GP-33 MOD "Ashfall" Grenade Launcher'),
 ('6a0c7828-4e94-4428-8363-c5431835a040',
  'GP-33 MOD "Copperhead" Grenade Launcher'),
 ('46691e82-b633-4d4a-b967-3fce30a0a01c', 'MK-4 Frag Grenade'),
 ('6f1674b1-fb58-4661-9114-f418862751d2', 'FS-9 LMG'),
 ('1f3400e1-0aa3-48bf-8595-38f4f4218df9', 'S-38 Pistol'),
 ('971a5272-a0f1-46a1-aee1-44955fe528e4', 'S-38 "One Empire" Pistol'),
 ('02d4cd2e-fa98-4086-aee1-6b2dfce8ea27', 'P4-AR Rifle'),
 (

## Connect to DB

In [45]:
import redis

db = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)

## Add each one to DB

In [46]:
DESCRIPTION_PARAMS = {
    'action': 'query',
    'prop': 'extracts',
    'exchars': '1200',
    'explaintext': '1',
    'format': 'json',
    'exintro': '1',
    'redirects': '1'
}

In [47]:
get_description = lambda title: next(iter(requests.get("https://starcitizen.tools/api.php", params=DESCRIPTION_PARAMS | {"titles": title}).json()["query"]["pages"].values())).get("extract", "").strip()

In [49]:
from parse import parse
for (uuid, name) in alive_it(r, force_tty=True):
    data = requests.get(f"https://api.star-citizen.wiki/api/v2/{API_NAME}/" + uuid, params={"locale": "en_EN", "include": "shops,shops.items"}).json()["data"]
    try:
        plaintext = parse(data, FORMAT)
        #print(plaintext)
    except Exception as e:
        print(f"Error parsing: {name}: {e}")
        continue
    db.json().set(f"docs:{name}", redis.commands.json.path.Path.root_path(), {
        "title": name,
        "content": plaintext,
        "description": get_description(name) or data["description"],
        "url": "https://starcitizen.tools/" + quote(name),
    })

on 9: Error parsing: MK-4 Frag Grenade: 'personal_weapon' is undefined          
on 29: Error parsing: Cyan QuikFlarePro: 'personal_weapon' is undefined         
on 30: Error parsing: Green QuikFlarePro: 'personal_weapon' is undefined        
on 31: Error parsing: Orange QuikFlarePro: 'personal_weapon' is undefined       
on 32: Error parsing: Pink QuikFlare: 'personal_weapon' is undefined            
on 35: Error parsing: Red QuikFlarePro: 'personal_weapon' is undefined          
on 36: Error parsing: Yellow QuikFlarePro: 'personal_weapon' is undefined       
on 142: Error parsing: MK-4 Frag Grenade: 'personal_weapon' is undefined        
on 173: Error parsing: Cyan QuikFlarePro: 'personal_weapon' is undefined        
on 174: Error parsing: Green QuikFlarePro: 'personal_weapon' is undefined       
on 175: Error parsing: Orange QuikFlarePro: 'personal_weapon' is undefined      
on 176: Error parsing: Red QuikFlarePro: 'personal_weapon' is undefined         
on 177: Error parsing: Yello

## Do a test query

In [50]:
from redis.commands.search.query import Query
from json import loads
index = db.ft("idx:docs")

In [51]:
res = index.search(Query('fs'))
for result in res.docs:
    print(loads(result.json))
if res.docs:
    print(loads(res.docs[0].json)["content"])

{'title': 'FS-9 LMG', 'content': '# Weapon: FS-9 LMG\nBehring designed the FS-9 to be an overwhelming battlefield force. This light machine gun features a full auto mode that makes it ideal for laying down covering fire or holding back advancing enemies. The FS-9 is also acclaimed for its ability to maintain accuracy while unleashing sustained fire.\n\n## Weapon Stats\nWeapon Type: Ballistic LMG\nMagazine: 120 rounds\nRange: 40 m\nFire Rate: 800 rpm\nDamage/Shot: 5.71\n\n## Attachments\nMagazine Slot (Size 1)\nOptics Slot (Size 1-2)\nBarrel Slot (Size 2)\nUnderbarrel Slot (Size 1-3)\n## Buyable at\nNot Buyable (loot only)\n', 'description': "The FS-9 LMG is a 5-mm light machine gun manufactured by Behring Applied Technology. With an effective range of 40 meters, the FS-9 is designed to be a dominant force on the battlefield. Behring's intention was to create a weapon that could provide overwhelming firepower. The full auto mode of the light machine gun is particularly useful for laying

# Embed Documents

In [52]:
import cohere
from dotenv import load_dotenv

load_dotenv()

co = cohere.Client()

Get the item descriptions

In [53]:
keys = sorted(db.keys("docs:*"))
descriptions = db.json().mget(keys, "$.description")
descriptions = [item for sublist in descriptions for item in sublist]

Grab them from cohere (ridiculously fast)

In [54]:
doc_embeddings = co.embed(descriptions, input_type="search_document", model="embed-english-v3.0").embeddings

Add the embeddings with a pipeline

In [55]:
pipeline = db.pipeline()
for key, embedding in zip(keys, doc_embeddings):
    pipeline.json().set(key, "$.description_embeddings", embedding)
pipeline.execute()
True

True

Run a test query

In [80]:
query_emb = co.embed(["Compare the ships in the 300 series"], input_type="search_query", model="embed-english-v3.0").embeddings[0]

In [81]:
import numpy as np

In [84]:
query = (
    Query('(*)=>[KNN 20 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .dialect(2)
)
result_docs = db.ft("idx:docs").search(query, { 'query_vector': np.array(query_emb, dtype=np.float32).tobytes() }).docs

print("Potentially relevant context:\n")
print("\n---\n\n".join([page.content for page in result_docs]))

Potentially relevant context:

# Ship Stats: 300i
Dimensions: 27m length, 17m width, 8m height
Emissions: IR: 6020, EM Idle: 6252, EM Max: 20643
Mass: 79677 kg
Cargo capacity: 8 SCU
Vehicle Inventory: 8
Personal Inventory: 0.65
Crew: Skeleton: 1, Combat: 0
Health: 9650
Shield HP: 3000 (From shield generator)
Speed: SCM: 204 m/s, Max: 1188 m/s, 0 to SCM: 2.154 s, 0 to Max: 12.544 s, SCM to 0: 4.902 s, Max to 0: 28.549 s
Fuel: Capacity: 105000L, Intake Rate: 12L/s, Usage: Main: 40L/s, Maneuvering: 48L/s, Retro: 41L/s, VTOL: 0L/s
Quantum: Speed: 148971800km/s, Spool Time: 5 s, Fuel Capacity: 680L, Range: 69387755102.04082 m
Agility: Pitch: 45°/s, Yaw: 40°/s, Roll: 150°/s, Acceleration: Main: 94.703 m/s², Retro: 41.611 m/s², VTOL: 0 m/s², Maneuvering: 193.504 m/s²
Armor: IR: 1, EM: 1, Cross Section: 1
Focus: Luxury Touring
Role: multi
Description: If you're going to travel the stars... why not do it in style? The 300i is Origin Jumpworks' premiere luxury spacecraft. It is a sleek, silver k

In [90]:
dir(result_docs[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'content',
 'description',
 'id',
 'payload',
 'title',
 'vector_score']

In [83]:
print("\n".join([r.vector_score for r in result_docs]))

0.465593218803
0.485395491123
0.536980688572
0.538796007633
0.592752993107
0.599588036537
0.63577067852
0.636471867561
0.636471867561
0.638890087605
